In [23]:
import trio_autoawait
%autoawait trio_autoawait.run_in_trio

In [1]:
# 
#
# Additional control:
# 
# trio_autoawait.stop()   # stop the Trio run
# trio_autoawait.start()  # start it -- restart if already running


In [20]:
import trio

In [21]:
await trio.sleep(0)

In [22]:
trio.current_time()

291593.39950810524

In [35]:
from subprocess import PIPE, STDOUT
import trio

async def ping():
    cmd = ['ping', 'google.com']
    async with await trio.open_process(cmd, stdout=PIPE, stderr=STDOUT) as process:
        while True:
            res = await process.stdout.receive_some()
            if not res:
                break
            print(res)

await ping()

/var/folders/6_/7xx0tpq16h9cn40mq4w5gjk80000gn/T/ipykernel_17638/1193476661.py:6: TrioDeprecationWarning: trio.open_process is deprecated since Trio 0.20.0; use trio.lowlevel.open_process instead (https://github.com/python-trio/trio/issues/1104)
  async with await trio.open_process(cmd, stdout=PIPE, stderr=STDOUT) as process:
/var/folders/6_/7xx0tpq16h9cn40mq4w5gjk80000gn/T/ipykernel_17638/1193476661.py:6: TrioDeprecationWarning: using trio.Process as an async context manager is deprecated since Trio 0.20.0; use run_process or nursery.start(run_process, ...) instead (https://github.com/python-trio/trio/issues/1104)
  async with await trio.open_process(cmd, stdout=PIPE, stderr=STDOUT) as process:


b'PING google.com (172.217.167.78): 56 data bytes\n64 bytes from 172.217.167.78: icmp_seq=0 ttl=118 time=17.542 ms\n'
b'64 bytes from 172.217.167.78: icmp_seq=1 ttl=118 time=17.261 ms\n'
b'64 bytes from 172.217.167.78: icmp_seq=2 ttl=118 time=22.511 ms\n'
b'64 bytes from 172.217.167.78: icmp_seq=3 ttl=118 time=23.885 ms\n'


KeyboardInterrupt: 

In [29]:
import multiprocessing
multiprocessing.freeze_support()

In [30]:
import functools
import multiprocessing
import trio
import trio_parallel


def loop(n):
    # Arbitrary CPU-bound work
    for _ in range(n):
        pass
    print("Loops completed:", n)


async def amain():
    t0 = trio.current_time()
    async with trio.open_nursery() as nursery:
        # Do CPU-bound work in parallel
        for i in [6, 7, 8] * 4:
            nursery.start_soon(trio_parallel.run_sync, loop, 10 ** i)
        # Event loop remains responsive
        t1 = trio.current_time()
        await trio.sleep(0)
        print("Scheduling latency:", trio.current_time() - t1)
        # This job could take far too long, make it cancellable!
        nursery.start_soon(
            functools.partial(
                trio_parallel.run_sync, loop, 10 ** 20, cancellable=True
            )
        )
        await trio.sleep(2)
        # Only explicitly cancellable jobs are killed on cancel
        nursery.cancel_scope.cancel()
    print("Total runtime:", trio.current_time() - t0)

In [36]:
await amain()

b'\n--- google.com ping statistics ---\n4 packets transmitted, 4 packets received, 0.0% packet loss\nround-trip min/avg/max/stddev = 17.261/20.300/23.885/2.940 ms\n'
Scheduling latency: 0.0024011669447645545
ERROR! Session/line number was not unique in database. History logging moved to new session 8


/Users/madhavajay/.local/share/virtualenvs/python_concurrency-2ltfH85i/lib/python3.10/site-packages/trio/_abc.py:261: TrioDeprecationWarning: trio.Process.aclose is deprecated since Trio 0.20.0; use run_process or nursery.start(run_process, ...) instead (https://github.com/python-trio/trio/issues/1104)
  await self.aclose()
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/madhavajay/.pyenv/versions/3.10.9/lib/python3.10/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Users/madhavajay/.pyenv/versions/3.10.9/lib/python3.10/multiprocessing/spawn.py", line 125, in _main
    prepare(preparation_data)
  File "/Users/madhavajay/.pyenv/versions/3.10.9/lib/python3.10/multiprocessing/spawn.py", line 236, in prepare
    _fixup_main_from_path(data['init_main_from_path'])
  File "/Users/madhavajay/.pyenv/versions/3.10.9/lib/python3.10/multiprocessing/spawn.py", line 287, in _fixup_main_from_path
    main_c